In [2]:
import numpy as np 
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
def filtro_kuwahara(img):
    img_extendida = np.pad(img,((2,2),(2,2),(0,0)),mode="edge")
    filas, columnas, dimension = img_extendida.shape
    img_filtro =  np.copy(img)
    for fil in range(2,filas-2):
        for col in range(2, columnas-2):
            a = img_extendida[fil-2:fil+1 , col-2:col+1]
            b = img_extendida[fil-2:fil+1 , col:col+3]
            c = img_extendida[fil:fil+3 , col:col+3] 
            d = img_extendida[fil:fil+3 , col-2:col+1]

            varianza_a = np.var(a[:,:,0])+np.var(a[:,:,1])+np.var(a[:,:,2])
            varianza_b = np.var(b[:,:,0])+np.var(b[:,:,1])+np.var(b[:,:,2])
            varianza_c = np.var(c[:,:,0])+np.var(c[:,:,1])+np.var(c[:,:,2])
            varianza_d = np.var(d[:,:,0])+np.var(d[:,:,1])+np.var(d[:,:,2])
            varianzas = [varianza_a,varianza_b,varianza_c,varianza_d]

            min_varianza = 999999
            cuadrante_min_varianza = a
            for cuadrante,varianza in zip([a,b,c,d],varianzas):
                if varianza < min_varianza:
                    min_varianza = varianza
                    cuadrante_min_varianza = cuadrante

            pixel_media_colores = np.mean(cuadrante_min_varianza, (0,1))

            img_filtro[fil-2,col-2] = pixel_media_colores
    
    return img_filtro
def encriptador(msg):
    dic = {
        'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13,
        'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26,
        ' ': 27, '.': 28, ',': 29, '?': 30, '!': 31, '¿': 32, '¡': 33, '(': 34, ')': 35, ':': 36, ';': 37, '-': 38, '“': 39,
        '‘': 40, 'á': 41, 'é': 42, 'í': 43, 'ó': 44, 'ú': 45, 'ü': 46, 'ñ': 47}
    

    lista_codigos = [dic[letra] for letra in (msg.lower())]
    lista_msg_encriptado = []
    for codigo in lista_codigos:
            for num in str(codigo):
                lista_msg_encriptado.append(int(num)+1)
            lista_msg_encriptado.append(-1)
    lista_msg_encriptado.append(0)
    msg_final = " ".join((str(digito) for digito in lista_msg_encriptado))
    return msg_final

nombre_img = input("Ingrese nombre de la imagen a utilizar como base: ")
imagen = Image.open(nombre_img)
img = np.array(imagen)
img_filtro = (filtro_kuwahara(img))

mensaje = input("Ingrese el mensaje a esconder: ")
msg_encriptado = (encriptador(mensaje)).split(" ")

#Recorrer la imagen
filas, columnas, dimension = img_filtro.shape
while len(msg_encriptado) > 0:
    for fil in range(1,filas,2):
        if len(msg_encriptado) == 0:
            break
        for col in range(1,columnas,2):
            if len(msg_encriptado) == 0:
                break
            canal_rojo = np.array([img_filtro[fil-1,col-1,0],img_filtro[fil-1,col,0],img_filtro[fil,col-1,0]])
            canal_verde = np.array([img_filtro[fil-1,col-1,1],img_filtro[fil-1,col,1],img_filtro[fil,col-1,1]])
            canal_azul = np.array([img_filtro[fil-1,col-1,2],img_filtro[fil-1,col,2],img_filtro[fil,col-1,2]])


            min_varianza = 99999999999
            canal_min_varianza = canal_rojo
            dimension_canal = 0    
            for dim, canal in enumerate([canal_rojo,canal_verde,canal_azul]):
                if np.var(canal) < min_varianza:
                    min_varianza = np.var(canal)
                    canal_min_varianza = canal
                    dimension_canal = dim
            
            prom_color = np.mean(canal_min_varianza)

            varianzas = [np.var(canal) for canal in [canal_rojo,canal_verde,canal_azul]]
        
            nuevo_valor = prom_color + float(msg_encriptado.pop(0))
            
            img_filtro[fil,col,dimension_canal] = nuevo_valor
            
nombre_img_filtro = input("Ingrese nombre del archivo de salida: ")
imagen2 = Image.fromarray(img_filtro)
imagen2.save(nombre_img_filtro)




KeyboardInterrupt: 